# General-purpose text embeddings on the IPU

This notebook describes how to use supported embeddings models to generate SOTA text embeddings on the IPU. You can use the:
* [E5 model](https://arxiv.org/pdf/2212.03533.pdf) (Emb**E**ddings from bidir**E**ctional **E**ncoder r**E**presentations) to generate text embeddings on the IPU.
* [Sentence Transformers MPNet Base V2](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) An embeddings model based on the MPNet base model.
* [Sentence-T5](https://arxiv.org/abs/2108.08877) runs on a pretrained T5 model encoder.

First, install the requirements for running this notebook:

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
! pip install sentence-transformers
! pip install --find-links https://download.pytorch.org/whl/cpu/torch_stable.html torch==1.13.1+cpu

Import the required modules for the notebook:

In [2]:
import os
import torch
import poptorch
import numpy as np
from tqdm.notebook import tqdm
import logging

We need to instantiate some global parameters that will be used to run the models.

The **micro batch size** (number of batches to process in parallel) is set to a smaller value of 2 due to its greater effect on device memory. 

We use on-IPU loops (**device iterations**) which iterate over a number of batches sequentially (where the iteration takes place on-device in one dataloader call), to extend the batch size for more throughput benefit (this is more efficient than loading smaller batches on the host a large number of times). 

Data parallelism is controlled by the **replication factor**, i.e. how many devices the batch sizes are replicated over. This value is set to `None` by default as it will be automatically determined by the `pod_type` of the machine being used. By default, the model itself requires 1 IPU to run, and if running on a IPU POD4 (4 IPU) machine, the replication factor is set to 4. Similarly, if running on an IPU POD16, it is set to 16. This can be overidden with a different value if needed, i.e., if `replication_factor=N` the model will be replicated over `N` IPUs as long as `N * n_ipu (number of IPUs a single instance of the model uses) <= total available IPUs`.

The total effective batch size for inference is calculated by:
```
effective_batch_size = replication_factor * device_iterations * micro_batch_size
```

The model itself, through model pipelining, can also be run over 4 IPUs (by setting `model_ipu` to 4), in which case the replication factor will be adjusted accordingly. The reason we might want to spread the model over more IPUs is to reduce the memory consumption of the model over a single machine (e.g., with 4 IPUs, we compute far fewer layers per IPU, while with 1 IPU, all model layers are on a single IPU) allowing for higher batch sizes to be used. This is particularly beneficial on an IPU POD16 machine, as the 4-IPU pipelined version of the model can be run at a higher effective batch size (with higher micro batch size) and achieve even higher overall batched throughput.

In [3]:
logger = logging.getLogger("")

n_ipu = os.getenv("NUM_AVAILABLE_IPU", 4)
model_ipu = 1
micro_batch_size = 2
device_iterations = 256
replication_factor = None

random_seed = 42

To run embeddings models, we will set up a generic IPU embeddings class which loads the pretrained model onto the IPU and runs the embedding pooling and normalisation stages in the forward pass along with the model.

In [4]:
import torch
import logging
from typing import Optional, List

from transformers import AutoModel
from optimum.graphcore.modeling_utils import to_pipelined
from optimum.graphcore import IPUConfig

logger = logging.getLogger("e5")

class IPUEmbeddingsModel(torch.nn.Module):
    def __init__(self, model, ipu_config: IPUConfig, fp16=True):
        super().__init__()
        self.encoder = to_pipelined(model, ipu_config)
        self.encoder = self.encoder.parallelize()
        if fp16: self.encoder = self.encoder.half()
    
    def pool(self, last_hidden_states: torch.Tensor, attention_mask: torch.Tensor, pool_type: str) -> torch.Tensor:
             
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    
        if pool_type == "avg":
            emb = last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
        elif pool_type == "cls":
            emb = last_hidden[:, 0]
        else:
            raise ValueError(f"pool_type {pool_type} not supported")

        return emb
    
    def forward(self, pool_type: str ='avg', **kwargs) -> torch.Tensor:
        outputs = self.encoder(**kwargs)
        
        embeds = self.pool(outputs.last_hidden_state, kwargs["attention_mask"], pool_type=pool_type)
        embeds = torch.nn.functional.normalize(embeds, p=2, dim=-1)

        return embeds

Before setting up and running each of the models, lets create a simple `infer` function which handles loading the batches from a dataloader and generating the embeddings from any model:

In [5]:
import time

def infer(model, dataloader):
    encoded_embeds = []
    with torch.no_grad():
        for batch_dict in tqdm(dataloader, desc='encoding'):
            lat = time.time()
            outputs = model(**batch_dict)
            lat = time.time() - lat
            
            encoded_embeds.append(outputs)
            print(f"batch len: {len(batch_dict['input_ids'])} | batch latency: {lat}s | per_sample: {lat/len(batch_dict['input_ids'])}s | throughput: {len(batch_dict['input_ids'])/lat} samples/s")
    
    return torch.cat(encoded_embeds, axis=0)

## Embeddings with E5-Large

In [42]:
import modeling_bert, modeling_mpnet, modeling_t5 # delete this cell once in source
# import optimum.graphcore # replace

First, use Transformers' `AutoConfig` to load the model config for the E5 large model. E5 uses a bidirectional encoder, essentially the encoder stage of a BERT model, to generate the trained embeddings. The config will define the architecture of the model, such as the number of encoder layers and size of the hidden dimension within the model.

We also need to tokenize the dataset, for this we define a custom transform function which applies the pre-trained tokenization for each model to the dataset. We will call this function when loading the function, to avoid loading multiple tokenized datasets at the same time.

We define some IPU specific configurations to get the most out of the model, the `get_ipu_config` function will set up the IPU config according to the model config, taking into consideration the defined number of IPUs for model parallelism, the number of IPUs available and batching configurations.

In [9]:
from config import get_ipu_config
from transformers import AutoConfig, AutoTokenizer, BatchEncoding

In [10]:
e5_model_name = 'intfloat/e5-large'
e5_tokenizer = AutoTokenizer.from_pretrained(e5_model_name)
e5_model_config = AutoConfig.from_pretrained(e5_model_name)
e5_model = AutoModel.from_pretrained(e5_model_name, config=e5_model_config)

def e5_transform_func(example) -> BatchEncoding:
    return e5_tokenizer(
        example['text'],
        max_length=e5_model_config.max_position_embeddings,
        padding="max_length",
        truncation=True
    )

e5_ipu_config = get_ipu_config(
    e5_model_config, n_ipu, model_ipu, device_iterations, replication_factor, random_seed)

/localdata/arsalanu/popsdk_venvs/3.2.1+1370/3.2.1+1370_poptorch/lib/python3.8/site-packages/optimum/graphcore/ipu_configuration.py:403: UserWarning: The "enable_half_first_order_momentum" parameter is deprecated
  warnings.warn('The "enable_half_first_order_momentum" parameter is deprecated')


## Embeddings with All-MPNet

For MPNet, we do the same for the pre-trained model.

In [11]:
mpnet_model_name = 'sentence-transformers/all-mpnet-base-v2'
mpnet_tokenizer = AutoTokenizer.from_pretrained(mpnet_model_name)
mpnet_model_config = AutoConfig.from_pretrained(mpnet_model_name)
mpnet_model = AutoModel.from_pretrained(mpnet_model_name, config=mpnet_model_config)

def mpnet_transform_func(example) -> BatchEncoding:
    return mpnet_tokenizer(
        example['text'],
        max_length=mpnet_model_config.max_position_embeddings,
        padding="max_length",
        truncation=True
    )
    
mpnet_ipu_config = get_ipu_config(
    mpnet_model_config, n_ipu, model_ipu, device_iterations, replication_factor, random_seed)

## Embeddings with Sentence-T5

Note that for T5, we need to use the `T5EncoderModel` instead of `AutoModel`. We must manually specify the encoder as T5 is an encoder-decoder model, and we don't want to load the decoder for embeddings generation. 

Transformers `AutoModel` supports a 1-to-1 mapping of architecture definitions to model types, and it will by default load the `T5Model` class. We can override this by directly importing and loading the pretrained model using `T5EncoderModel`.

In [22]:
from transformers.models.t5.modeling_t5 import T5EncoderModel

t5_model_name = 'sentence-transformers/sentence-t5-base'
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)
t5_model_config = AutoConfig.from_pretrained(t5_model_name)
t5_model = T5EncoderModel.from_pretrained(t5_model_name, config=t5_model_config)

def t5_transform_func(example) -> BatchEncoding:
    return t5_tokenizer(
        example['text'],
        max_length=t5_model_config.n_positions,
        padding="max_length",
        truncation=True
    )

t5_ipu_config = get_ipu_config(
    t5_model_config, n_ipu, model_ipu, device_iterations, replication_factor, random_seed)

`replicated_tensor_sharding` is not used when `replication_factor=1`


## Creating the embeddings model

Then simply pass the model and IPU config to the `IPUEmbeddingsModel` class to initialise it. To run the model on the IPU, the IPU config needs to be first converted to an `IPUOptions` class and passed alongside the model to the `poptorch.inferenceModel` wrapper.

We'll wrap this behaviour into a simple function so we can iteratively run all three models and initialise the `poptorch.Dataloader` to create an IPU-ready batched dataloader. We pass an arbitrary call to the model using the first batch to ensure we have compiled the model executable (or loaded the already compiled executable).

In [16]:
from transformers import default_data_collator as data_collator

def create_model(model, ipu_config, dataset):
    model = IPUEmbeddingsModel(model, ipu_config)

    ipu_options = ipu_config.to_options(for_inference=True)
    model = poptorch.inferenceModel(model, ipu_options)

    dataloader = poptorch.DataLoader(
        ipu_options,
        dataset['train'],
        batch_size=micro_batch_size,
        shuffle=False,
        drop_last=True,
        num_workers=2,
        collate_fn=data_collator
    )

    model(**next(iter(dataloader)))
    return model, dataloader

Lets load a dataset we'll use to try out the models. Using the Hugging Face `datasets` library we can load a pre-existing dataset from the Hugging Face Hub. In this case, lets use the `rotten_tomatoes` film review dataset. Later in the notebook, we will use this dataset to perform create a basic semantic search functionality.

In [17]:
from datasets import Dataset, load_dataset
dataset = load_dataset("rotten_tomatoes")

Found cached dataset rotten_tomatoes (/home/arsalanu/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

## Run the E5 model

The dataset first needs to be tokenized using the pre-trained tokenizer for each model, we can use the `map()` method to tokenize each of the inputs of the dataset using the model-specific transform function. Then we can convert the Hugging Face Arrow format dataset to a Pytorch ready dataset with `set_format` which converts the tokenized inputs into tensors.

To run the model, Simply call the `infer` function we created earlier to generate embeddings for the full dataset. 

In [19]:
tokenized_dataset = dataset.map(e5_transform_func, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

model, dataloader = create_model(e5_model, e5_ipu_config, tokenized_dataset)

e5_data_embeddings = infer(model, dataloader)

model.detachFromDevice()

Loading cached processed dataset at /home/arsalanu/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-7112dabf7b658c8c.arrow
Loading cached processed dataset at /home/arsalanu/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-acd391fc56701cfa.arrow
Loading cached processed dataset at /home/arsalanu/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-5819b9c1eb01ab55.arrow


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Graph compilation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00]


encoding:   0%|          | 0/4 [00:00<?, ?it/s]

batch len: 2048 | batch latency: 2.401418685913086s | per_sample: 0.001172567717730999s | throughput: 852.8292096724873 samples/s
batch len: 2048 | batch latency: 2.3685898780822754s | per_sample: 0.001156538026407361s | throughput: 864.6494772907497 samples/s
batch len: 2048 | batch latency: 2.368685245513916s | per_sample: 0.0011565845925360918s | throughput: 864.61466498292 samples/s
batch len: 2048 | batch latency: 2.3684709072113037s | per_sample: 0.0011564799351617694s | throughput: 864.6929095748809 samples/s
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Run the All-MPNet model

In [20]:
tokenized_dataset = dataset.map(mpnet_transform_func, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

model, dataloader = create_model(mpnet_model, mpnet_ipu_config, tokenized_dataset)

mpnet_data_embeddings = infer(model, dataloader)

model.detachFromDevice()

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Graph compilation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:30<00:00]


encoding:   0%|          | 0/4 [00:00<?, ?it/s]

batch len: 2048 | batch latency: 1.6170854568481445s | per_sample: 0.0007895925082266331s | throughput: 1266.4760488241293 samples/s
batch len: 2048 | batch latency: 1.6077561378479004s | per_sample: 0.0007850371766835451s | throughput: 1273.8250234524985 samples/s
batch len: 2048 | batch latency: 1.6077404022216797s | per_sample: 0.0007850294932723045s | throughput: 1273.8374909095655 samples/s
batch len: 2048 | batch latency: 1.6073262691497803s | per_sample: 0.0007848272798582911s | throughput: 1274.1656994651876 samples/s
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...


## Run the Sentence-T5 model

In [26]:
tokenized_dataset = dataset.map(t5_transform_func, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

model, dataloader = create_model(t5_model, t5_ipu_config, tokenized_dataset)

t5_data_embeddings = infer(model, dataloader)

model.detachFromDevice()

Loading cached processed dataset at /home/arsalanu/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-caf138e417debee9.arrow
Loading cached processed dataset at /home/arsalanu/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-307125742b434b14.arrow
Loading cached processed dataset at /home/arsalanu/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-97055b378d319084.arrow


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Graph compilation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:14<00:00]


encoding:   0%|          | 0/4 [00:00<?, ?it/s]

batch len: 2048 | batch latency: 1.1848907470703125s | per_sample: 0.000578559935092926s | throughput: 1728.4293974476195 samples/s
batch len: 2048 | batch latency: 1.1810739040374756s | per_sample: 0.0005766962422057986s | throughput: 1734.015113701993 samples/s
batch len: 2048 | batch latency: 1.1660363674163818s | per_sample: 0.0005693536950275302s | throughput: 1756.3774657713368 samples/s
batch len: 2048 | batch latency: 1.1653735637664795s | per_sample: 0.0005690300604328513s | throughput: 1757.376401589957 samples/s


The embeddings for a single sequence represent low-dimensional numerical representations of the word-level and sentence-level context for each token. These pre-trained embeddings can be used in applications like embedding retrieval for recommender systems, or semantic search for query-matching using cosine-similarity. Both of these use cases take advantage of the generated embeddings space, by performing a relative comparison of the user input sequence embeddings using some proximity metric.

We'll use the open source `sentence_transformers` library which provides utilities for embeddings tasks to perform semantic search on a user query to retrieve the most similar sequences from the dataset to the query. This is a helpful utility for making, for example, more responsive FAQs.

## Semantic search with generated embeddings

Using the `rotten_tomatoes` dataset, lets create a simple similarity search engine using `sentence_transformers` semantic search function, which uses cosine similarity to retrieve close-proximity sentences from a given set of embeddings to a given query. We have already generated embeddings for the dataset, so the next step is to do the same with a given query and perform the search.

First, to process the query, we need to tokenize it and convert it to a single-batch input for the model. This has been wrapped into a simple function which tokenizes and prepares a dictionary of model inputs (`input_ids`, `attention_mask`, etc.,) to which we just need to pass a string.

In [35]:
def prepare_query(query: str):
    t_query = mpnet_tokenizer(
            query,
            max_length=mpnet_model_config.max_position_embeddings,
            padding="max_length",
            truncation=True
        )

    return {k: torch.as_tensor([t_query[k]]) for k in t_query}

Next, to perform inference with a single input (i.e., effective batch size of 1) we re-instantiate the model (for this example, lets use the All-MPNet model) by setting all device batching, replication and micro batch-size to 1 and re-compile the model. The change in batch size necessitates a recompilation, since the input shape to the model has been changed. We will follow the steps to initiate the model outlined earlier in the notebook, with the only change being setting the `get_ipu_config` function to have all batching turned off.

In [36]:
mpnet_infer_ipu_config = get_ipu_config(
    mpnet_model_config, n_ipu, model_ipu=1, device_iterations=1, replication_factor=1, random_seed=random_seed)

model = IPUEmbeddingsModel(mpnet_model, mpnet_infer_ipu_config)
model = poptorch.inferenceModel(model, mpnet_infer_ipu_config.to_options(for_inference=True))

model(**prepare_query("Running once to compile"))

Graph compilation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:53<00:00]


tensor([[-2.7069e-02,  3.1342e-02, -2.2141e-02, -5.3741e-02,  7.3280e-03,
          5.1941e-02,  3.7842e-02, -2.4475e-02, -1.6937e-02,  2.7027e-03,
          7.0435e-02,  7.4463e-02,  3.8635e-02, -4.4067e-02,  2.1652e-02,
          7.5439e-02, -6.2904e-03,  2.0996e-02, -5.2063e-02, -5.7602e-03,
         -4.4647e-02, -2.0355e-02,  5.6152e-02, -3.8605e-03, -3.6560e-02,
         -3.6102e-02,  1.8204e-02, -2.0172e-02,  6.9351e-03,  2.7908e-02,
         -2.9793e-03,  1.1169e-02,  6.1455e-03, -3.3112e-02,  1.3113e-06,
          7.3547e-03,  7.8247e-02,  3.4882e-02, -4.5380e-02, -2.1057e-03,
         -7.1869e-03,  6.4575e-02,  2.5711e-02,  6.8115e-02,  1.2619e-02,
         -5.2929e-04, -4.1718e-02, -6.0883e-02, -1.8021e-02,  5.1165e-04,
          1.2749e-02, -6.2042e-02, -2.5158e-03,  7.4921e-03,  1.2070e-02,
         -4.1870e-02,  1.0063e-02, -3.4428e-03, -1.6159e-02,  4.7241e-02,
          4.4312e-02,  6.0944e-02,  9.2850e-03, -3.3665e-03, -2.1423e-02,
         -2.3529e-02,  3.4790e-02, -1.

Finally, we can use the model to embed a single query, and perform a semantic search across the full dataset embeddings to retrieve highly relevant reviews to the query.

In [38]:
from sentence_transformers.util import semantic_search

query = "Strongly disliked this action movie"

query_embeddings = model(**prepare_query(query))
hits = semantic_search(query_embeddings.float(), mpnet_data_embeddings.float(), top_k=10)

print(f"\n SEARCH QUERY: {query}")
for n, res in enumerate(hits[0]):
    print(f"\n Result (rank {n+1}) | Score: {res['score']} | Text: {dataset['train']['text'][res['corpus_id']]} ")


 SEARCH QUERY: Strongly disliked this action movie

 Result (rank 1) | Score: 0.7619101405143738 | Text: it's a bad action movie because there's no rooting interest and the spectacle is grotesque and boring . 

 Result (rank 2) | Score: 0.6812450289726257 | Text: i hate this movie 

 Result (rank 3) | Score: 0.6624367237091064 | Text: features nonsensical and laughable plotting , wooden performances , ineptly directed action sequences and some of the worst dialogue in recent memory . 

 Result (rank 4) | Score: 0.6615514755249023 | Text: 'this movie sucks . ' 

 Result (rank 5) | Score: 0.6549775004386902 | Text: it is a comedy that's not very funny and an action movie that is not very thrilling ( and an uneasy alliance , at that ) . 

 Result (rank 6) | Score: 0.6511269211769104 | Text: the worst film of the year . 

 Result (rank 7) | Score: 0.6448710560798645 | Text: the acting is amateurish , the cinematography is atrocious , the direction is clumsy , the writing is insipid and th